In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
import skimage as sk
import skimage.io as io
import matplotlib
%matplotlib inline
from skimage.util import img_as_ubyte
import utils, morph
from main import *

In [10]:
from scipy.spatial import Delaunay
from scipy.interpolate import interp2d

In [11]:
from pathlib import Path
data = Path("input")
data.mkdir(parents=True, exist_ok=True)

In [12]:
# matplotlib.use("TkAgg")
# matplotlib.use("Tk")
# matplotlib.interactive(True)

In [21]:
me = utils.read_img("src_images/me.jpg")
me.shape
# io.imshow(me)

(2640, 1980, 3)

In [16]:
matplotlib.use("TkAgg")
utils.align_img("src_images/me.jpg")

Please select the eyes for alignment.
Please select 2 points in image.
float64


In [17]:
utils.align_img("src_images/iu_4.jpg")

Please select the eyes for alignment.
Please select 2 points in image.
float64


In [20]:
iu = utils.read_img("src_images/iu_4.jpg")
iu.shape

(1071, 900, 3)